# Explore here

In [1]:
import numpy as np, random
import pandas as pd
import dotenv
df_raw = pd.read_csv('C:/Users/Josue/4GA.DataScience/data/raw/Spam.csv')


In [2]:
df_raw.info()
print('\n','---------------------------------------------------------------------------------------','\n',)
df_raw.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB

 --------------------------------------------------------------------------------------- 



(2999, 2)

In [3]:
#Pasamos un filtro para duplicados, aunque parece que el archivo en cuanto a datos faltantes esta bien!
df_raw= df_raw.drop_duplicates()
df_raw=df_raw.reset_index(inplace= False, drop = True)
df_raw.info


<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...     True
1                                https://www.hvper.com/     True
2                    https://briefingday.com/m/v4n3i4f3     True
3      https://briefingday.com/n/20200618/m#commentform    False
4                           https://briefingday.com/fan     True
...                                                 ...      ...
2364  https://www.theverge.com/2020/6/29/21306889/di...    False
2365  https://www.smartcitiesworld.net/news/news/dee...    False
2366  https://techcrunch.com/2019/07/04/an-optimisti...    False
2367  https://www.technologyreview.com/2019/12/20/13...    False
2368       https://www.bbc.com/news/technology-51018758    False

[2369 rows x 2 columns]>

Entendemos que la segunda columna es la variable target, columna que indica si la primera es spam o no.
Como no hay un conjunto para usar y predecir mediante el modelo entrenado creamos el modelo y lo guardamos.

In [ ]:
#1 Transformamos los datos. La columna descriptiva tendremos que eliminar todo lo que no sean letras. Y sintaxis
#De la estructura de un vinculo o enlace web, como wwww/https/.com/es/info... etc
#La segunda columna la factorizamos , pasamos el valor binario booleano true , false a 0 o 1.
#Si en el paso 1 no lo hacemos correctamente el Modelo SVC 'Técnicamente hablando' tendria fuga de datos
#Ya que la estructura de una pagina web es muy reconocible y se facilitaria demasiado al modelo saber si es 
#Spam

In [4]:
#Pasamos la columna Booleana a 1 o 0:
df_raw["is_spam"] = df_raw["is_spam"].apply(lambda x: 1 if x else 0).astype(int)
df_raw.head(6)

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1
5,https://www.brookings.edu/interactives/reopeni...,0


In [5]:
import re
from urllib.parse import urlparse
import pandas as pd

def filtrotexto(text):
    text = re.sub(r'[^a-z ]', " ", text.lower())
    text = re.sub(r'\s+[a-z]\s+', " ", text)
    text = re.sub(r'^[a-z]\s+', " ", text)
    text = re.sub(r'\s+[a-z]$', " ", text)
    text = re.sub(r'\s+', " ", text).strip()
    return text.split()

def limpiarurl_mejorado(url):
    parsed_url = urlparse(url)
    cleaned_parts = []

    
    if parsed_url.scheme:
        cleaned_parts.extend(preprocess_url_part(parsed_url.scheme.replace('s', ''))) 
    if parsed_url.netloc:
        netloc_parts = parsed_url.netloc.split('.')
        for part in netloc_parts:
            sub_parts = part.split('-')
            for sub_part in sub_parts:
                cleaned_parts.extend(preprocess_url_part(sub_part))    
    if parsed_url.path:
        path_parts = parsed_url.path.split('/')
        for part in path_parts:
            cleaned_parts.extend(preprocess_url_part(part))
    return [part for part in cleaned_parts if part] # Remove empty strings
df_raw["url"] = df_raw["url"].apply(filtrotexto)
print(df_raw.head(6))

                                                 url  is_spam
0  [https, briefingday, us, list, manage, com, un...        1
1                           [https, www, hvper, com]        1
2                    [https, briefingday, com, v, i]        1
3          [https, briefingday, com, m, commentform]        0
4                     [https, briefingday, com, fan]        1
5  [https, www, brookings, edu, interactives, reo...        0


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib
import os 

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_raw['url'].apply(lambda x: ' '.join(x)))
y = df_raw['is_spam']
model = SVC()
kf = KFold(n_splits=3, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kf)
print(f'Puntuaciones de validación cruzada: {scores}')
print(f'Puntuación media de validación cruzada: {scores.mean()}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
model_path = os.path.join('models', 'C:/Users/Josue/4GA.DataScience/models/ModelSVC.joblib')
vectorizer_path = os.path.join('models', 'C:/Users/Josue/4GA.DataScience/models/vectorizer.joblib')
os.makedirs('models', exist_ok=True) 
joblib.dump(model, model_path)
joblib.dump(vectorizer, vectorizer_path)

Puntuaciones de validación cruzada: [0.93924051 0.92405063 0.92141952]
Puntuación media de validación cruzada: 0.9282368858727333
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       435
           1       1.00      0.49      0.66        39

    accuracy                           0.96       474
   macro avg       0.98      0.74      0.82       474
weighted avg       0.96      0.96      0.95       474



['C:/Users/Josue/4GA.DataScience/models/vectorizer.joblib']

In [10]:
import pandas as pd
import re
from urllib.parse import urlparse
import joblib
#En este apartado probaremos el modelo entrenado con un dataframe de prueba con paginas de deportes y paginas inventadasa a mano.
# DataFrame de prueba
df_test = pd.DataFrame({
    'url': [
        'www.marca.com',
        'www.qdqwdsadqd1224232343419812QQSDSF.com',
        'www.as.com',
        'www.qdqwdsadqd1224232343419812QQSDSF.com',
        'www.sport.es',
        'www.mundodeportivo.com',
        'www.qdqwdsadqd1224232343419812QQSDSF.com',
        'www.espn.com',
        'https://bit.ly/xyz123',
        'www.foxsports.com',
        'https://cutt.ly/abc456',
        'www.qdqwdsadqd1224232343419812QQSDSF.com',
        'www.cbsports.com',
        'https://tinyurl.com/def789',
        'www.qdqwdsadqd1224232343419812QQSDSF.com',
    ]
})
df_test['is_spam'] = 2  # Inicializar con 2 para verificar sobrescritura

# Funciones de preprocesamiento
def filtrotexto(text):
    text = re.sub(r'[^a-z ]', " ", text.lower())
    text = re.sub(r'\s+[a-z]\s+', " ", text)
    text = re.sub(r'^[a-z]\s+', " ", text)
    text = re.sub(r'\s+[a-z]$', " ", text)
    text = re.sub(r'\s+', " ", text).strip()
    return text.split()

def preprocess_url_part(part):
    return filtrotexto(part)

def limpiarurl_mejorado(url):
    parsed_url = urlparse(url)
    cleaned_parts = []
    if parsed_url.scheme:
        cleaned_parts.extend(preprocess_url_part(parsed_url.scheme.replace('s', '')))
    if parsed_url.netloc:
        netloc_parts = parsed_url.netloc.split('.')
        for part in netloc_parts:
            sub_parts = part.split('-')
            for sub_part in sub_parts:
                cleaned_parts.extend(preprocess_url_part(sub_part))
    if parsed_url.path:
        path_parts = parsed_url.path.split('/')
        for part in path_parts:
            cleaned_parts.extend(preprocess_url_part(part))
    return [part for part in cleaned_parts if part]

def unir_lista(lista):
    return ' '.join(lista)

# Preprocesamiento de datos de prueba
df_test["url"] = df_test["url"].apply(limpiarurl_mejorado)
df_test["url"] = df_test["url"].apply(unir_lista)

# Carga de modelo y vectorizador
model_path = 'C:/Users/Josue/4GA.DataScience/models/ModelSVC.joblib'
vectorizer_path = 'C:/Users/Josue/4GA.DataScience/models/vectorizer.joblib'
model = joblib.load(model_path)
vectorizer = joblib.load(vectorizer_path)

# Vectorización y predicción
X_test = vectorizer.transform(df_test['url'])
y_pred = model.predict(X_test)

# Resultados
print("DataFrame antes de asignar predicciones:")
print(df_test)
df_test['is_spam'] = y_pred
print("\nDataFrame después de asignar predicciones:")
print(df_test)
print("\nValores únicos en la columna is_spam:")
print(df_test['is_spam'].unique())

DataFrame antes de asignar predicciones:
                          url  is_spam
0               www marca com        2
1   www qdqwdsadqd qqsdsf com        2
2                  www as com        2
3   www qdqwdsadqd qqsdsf com        2
4                www sport es        2
5      www mundodeportivo com        2
6   www qdqwdsadqd qqsdsf com        2
7                www espn com        2
8             http bit ly xyz        2
9           www foxsports com        2
10           http cutt ly abc        2
11  www qdqwdsadqd qqsdsf com        2
12           www cbsports com        2
13       http tinyurl com def        2
14  www qdqwdsadqd qqsdsf com        2

DataFrame después de asignar predicciones:
                          url  is_spam
0               www marca com        0
1   www qdqwdsadqd qqsdsf com        0
2                  www as com        0
3   www qdqwdsadqd qqsdsf com        0
4                www sport es        0
5      www mundodeportivo com        0
6   www qdqwdsadqd